# Viabilidade técnica de extração de informação dos Comprovantes PDF 

## Hipótese

A principal hipose é que *Podemos Extrair* o custo e o CFP/CNPJ a partir dos comprovantes submetidos pelos proponentes.

O *Resultado Esperado* é que, para cada comprovante submetido, conseguimos extrair o CNPJ/CPF do fornecedor, além do custo empenhado pela nota fiscal

*Teremos sucesso quando* conseguirmos extrair tais informações de pelo menos 50% dos comprovantes. 

## Solução Proposta

A solução proposta usa a OCR Tesseract (https://github.com/tesseract-ocr) para a extração de texto a partir de imagens. Para isso a seguinte sequencia de processamento foi necessário:
* [ x ] Querbrar os pdf para ter um comprovante por pdf
* [x] Converter os arquivos pdf em arquivos de imagem
* [x] Fazer o mínimo de processamento de imagem para melhorar a qualidade dos comprovantes
* [x] Extrair o texto a partir da imagem, utilizando o Tesseract
* [x] Extrair por meio de regex o CNPJ/CPF presente no arquivo texto extraído
* [ ] Extrair por meio de regex o valor empenhado na nota fiscal

### 1. Converter pdf em arquivo texto

In [ ]:
import argparse
import cv2
import pytesseract
import os
import glob
from PIL import Image, ImageEnhance, ImageFilter

foldernames = sorted(glob.glob('../pdfs/'))
len(foldernames)
for f in foldernames:
    foldersearch= f + "*.jpg"
    filenames = sorted(glob.glob(foldersearch)) 
    i = 0
    for file in filenames:
        print(file)
        im = Image.open(file)
        im = im.filter(ImageFilter.MedianFilter())
        enhancer = ImageEnhance.Contrast(im)
        im = enhancer.enhance(2)
        im = im.convert('1')
        im.save(file)
        text = pytesseract.image_to_string(Image.open(file), lang='por')
        
#         os.remove(file)
        print(text)
        text_file = open( f+"%s.txt" % i , "w")
        i += 1
        text_file.write(text)
        text_file.close()

../pdfs/041 correios.jpg


### 2. Extrair CNPJ a partir do texto

In [ ]:
import re
import argparse
import cv2
import pytesseract
import os
import glob
from PIL import Image, ImageEnhance, ImageFilter

p = re.compile('\d{2}\.\d{3}\.\d{3}\/\d{4}-\d{2}')
foldernames = sorted(glob.glob('../pdfs/'))
print(len(foldernames))
i = 0
for f in foldernames:
    foldersearch= f + "*.txt"
    filenames = sorted(glob.glob(foldersearch)) 
    for file in filenames:
        text_file = open( file , "r")
        for line in text_file:
            s = p.match(line)
            if s:
                print(s.group(), i)
                i += 1

### 3. Extrair Valor empenhado a partir do texto

## Resultados e Analise

Para um primeiro experimento, conseguimos extrair de forma automatizada somente 30% dos CNPJs dos comprovantes, o que faz com que nao tenhamos atingido nossa meta de sucesso. Logo, há necessidade de evoluir principalmente o algoritmo de processamento de imagem para melhorar a qualidade dos comprovantes

## Próximos passos

* [] Validar padrões regex nos arquivos pdf
* [] extrair preços, a partir de regex
* [] melhorar o processamento de imagem (filtros, contraste, orientação do scanner)
* [] testar algoritmos de extraçao de texto a partir de arquivos pdf nativos (&& apt-get -y install poppler-utils )
